# lazy_property
就是代理模式中的虚拟代理
类似单例的懒汉模式

https://stackoverflow.com/questions/24704147/python-what-is-a-lazy-property

http://python.jobbole.com/85553/

It is a property decorator that gets out of the way after the first call. It allows you to auto-cache a computed value.

## 普通property

先看普通property(知识点:描述符类)

In [1]:
class Test:
    def __init__(self):
        self.x='foo'
        self.y='bar'
        self._resouce=None
    @property
    def resource(self):
        print('initializing self._resource which is: {}'.format(self._resouce))
         # 代价大的操作，初始化数据库什么的
        self._resouce=tuple(range(10)) 
        return self._resouce
    @resource.setter
    def set_resource(self,val):
        self._resouce = val

In [2]:
t = Test()

In [3]:
Test.resource

In [4]:
t.resource

initializing self._resource which is: None


(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [26]:
t.__dict__

{'x': 'foo', 'y': 'bar', '_resouce': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)}

In [20]:
t.resource = 1

AttributeError: can't set attribute

每次调用，都会初始化资源，如果资源很重，挺麻烦

In [434]:
t.resource

initializing self._resource which is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [435]:
t.resource

initializing self._resource which is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

## lazy_property

核心就是setattr，让1次属性查找时，不再走__get__即描述符属性流程，而是直接从`instance.__dict__`中找到

也类似thread_once这种
    

In [2]:
class LazyProperty(object):
    def __init__(self,method):
        self.method = method
        self.method_name = method.__name__
    # __get__是描述符的用法
    def __get__(self,instance,owner):
        # 必须有instance，意味Text.resource 永远return None
        if not instance:
            return None
        # self.method是谁? 是resource(self)
        # 所以调用原来的resource(self)时，需要传入instance作为参数(即self)
        value = self.method(instance)
#         print('set %s = %(self.method_name,value))
        # 得到的值和属性名称，再设置为instance的1个属性,下次查找instance的method_name属性时，不会再调__get__了，因此
        # value只会保存1次.
        setattr(instance,self.method_name,value)
        

其实就是装饰器实现，解释:


```
resource  = LazyProperty(resource)
1. resoucre变成了1个LazyProperty的实例，输入是原来的resource成员方法;
2. LazyProperty定义了__get__，是1个描述符对象，这个对象的实例(即resource)作为Text的1个类成员，意味着它成为属性
可以进行Text.resource = xxx的操作. __get__定义了 读取Text.resource时的动作.
```

In [3]:
class Test:
    def __init__(self):
        self.x='foo'
        self.y='bar'
        self._resouce=None
    @LazyProperty
    def resource(self):
        print('initializing self._resource which is: {}'.format(self._resouce))
         # 代价大的操作，初始化数据库什么的
        self._resouce=tuple(range(10)) 
        return self._resouce


In [4]:
t= Test()

In [5]:
t.x

'foo'

In [6]:
t.y

'bar'

In [7]:
t.resource

initializing self._resource which is: None


核心就是它，第1次访问为None,但是已经create了，第2次访问 (实际要多lazy，取决于实现)t.resource时，不再创建，直接返回.

In [8]:
t.resource

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [9]:
t.resource

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

## 应用

### werkzurg里的cached_property

### blinker直接定义了lazy_property

```py
class lazy_property(object):
    """A @property that is only evaluated once."""

    def __init__(self, deferred):
        self._deferred = deferred
        self.__doc__ = deferred.__doc__

    def __get__(self, obj, cls):
        if obj is None:
            return self
        value = self._deferred(obj)
        setattr(obj, self._deferred.__name__, value)
        return value
```

使用的地方，receiver_connected是1个属性，也是1个Signal对象

```py
class Signal(object):
    """A notification emitter."""

    #: An :obj:`ANY` convenience synonym, allows ``Signal.ANY``
    #: without an additional import.
    ANY = ANY

    @lazy_property
    def receiver_connected(self):
        """Emitted after each :meth:`connect`.

        The signal sender is the signal instance, and the :meth:`connect`
        arguments are passed through: *receiver*, *sender*, and *weak*.

        .. versionadded:: 1.2

        """
        return Signal(doc="Emitted after a receiver connects.")
```